# <span style='color:SteelBlue'>P7 - Implémentez un modèle  de scoring (part 2)</span>

![Logo pret_a_depenser](https://user.oc-static.com/upload/2023/03/22/16794938722698_Data%20Scientist-P7-01-banner.png)

# <span style='background:white; color:black'>Sommaire</span>

**Introduction**

Importations des librairies<br>

Paramètres d'affichage<br>

Fonctions <br>

**Etape 1 : ...**

## <span style='background:red'>MAJ à faire</span>

**Conclusion**

# <span style='background:blue'>Introduction</span>

L'entreprise **"Prêt à dépenser"** souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un **algorithme de classification** en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).
___
**Missions**<br>
1/ Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.

2/ Analyser les features qui contribuent le plus au modèle, d’une manière générale (feature importance globale) et au niveau d’un client (feature importance locale), afin, dans un soucis de transparence, de permettre à un chargé d’études de mieux comprendre le score attribué par le modèle.

3/ Mettre en production le modèle de scoring de prédiction à l’aide d’une API et réaliser une interface de test de cette API.

4/ Mettre en oeuvre une approche globale MLOps de bout en bout, du tracking des expérimentations à l’analyse en production du data drift.
___
Ce notebook presente la 2ème partie concernant la construction du modèle de scoring.

___

**Source des datasets**<br>
Home Credit est une institution financière internationale de prêts à la consommation. Elle nous fournit un jeu de données comportant des informations sur les clients, les crédits qu'ils ont contractés, leur revenus, etc. Ces données sont utilisées pour construire un modèle de scoring de crédit pour prédire la probabilité de capacité de remboursement d'un client, et donc de déterminer si un crédit doit lui être accordé ou non.

Source : [Home Credit Default Risk](https://www.kaggle.com/competitions/home-credit-default-risk/overview) sur Kaggle.com<br>

10 fichiers CSV :
- application_train.csv
- application_test.csv
- bureau.csv
- bureau_balance.csv
- credit_card_balance.csv
- HomeCredit_columns_description.csv
- installments_payments.csv
- POS_CASH_balance.csv
- previous_application.csv
- sample_submission.csv

Voici le diagramme entité-association des données (diagramme ERD) :

![mappage_dataset](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

# <span style='background:grey'>Importations des librairies utilisees dans ce notebook</span>

In [1]:
# Librairies générales
import sys
import os
import time
from datetime import datetime
import pickle
from IPython.display import display
import warnings
import gc

# Data science
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# machine learning (général)
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
from sklearn.preprocessing import StandardScaler

# machine learning (modèles)
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Imbalanced-learn (Gestion des classes déséquilibrées)
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# Encodage des variables
from category_encoders.target_encoder import TargetEncoder

# Optimisation des hyperparametres Bayesienne
from skopt import BayesSearchCV

# mlflow
import mlflow
import mlflow.sklearn

# Affichage plot dans le notebook
%matplotlib inline

# Versions
print("Version des librairies utilisees :")
print("Python        : " + sys.version)
print("Matplotlib    : " + mpl.__version__)
print("Missingno     : " + msno.__version__)
print("Numpy         : " + np.__version__)
print("Pandas        : " + pd.__version__)
print("Scikit-learn  : " + sklearn.__version__)
print("Seaborn       : " + sns.__version__)

# Afficher heure lancement
maintenant = datetime.now().isoformat()
print("\nCode lance le : " + maintenant)

# Enregistrer l'heure de debut
start_time = time.time()

# Activer la verification PEP8
print("")
%load_ext pycodestyle_magic
%pycodestyle_on

Version des librairies utilisees :
Python        : 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]
Matplotlib    : 3.8.3
Missingno     : 0.5.2
Numpy         : 1.26.4
Pandas        : 2.2.1
Scikit-learn  : 1.4.1.post1
Seaborn       : 0.13.2

Code lance le : 2024-03-02T11:57:36.793806



# <span style='background:grey'>Paramètres d'affichage</span>

In [2]:
# nombre ligne, col et chiffre apres la virgule
pd.set_option("display.max_row", 50)
pd.set_option("display.max_column", 150)
pd.set_option("display.precision", 2)

# Ignore warnings
warnings.filterwarnings('ignore', category=UserWarning, module='distutils')
warnings.filterwarnings(
    'ignore',
    category=UserWarning,
    module='_distutils_hack'
)

# Graine aléatoire
random_seed = 42

# <span style='background:grey'>Fonctions</span>

In [3]:
def calculer_duree_notebook(start_time):
    """
    Cette procédure calcule et affiche la durée totale depuis un temps de
    début donné.

    Args:
        start_time (float): Le temps de début en secondes depuis l'époque.

    Returns:
        None
    """
    # Calculer la durée totale et convertir en minutes et secondes
    minutes, seconds = divmod(time.time() - start_time, 60)

    # Afficher la durée totale
    print(f"Durée execution notebook : {int(minutes)} min {int(seconds)} sec")

In [4]:
def calcul_valeur_manquante(df):
    """
    Calcule le nombre total de valeurs manquantes dans le DataFrame.

    :param df: DataFrame à analyser
    :return: nombre total de valeurs manquantes, nombre total de données,
    pourcentage de valeurs manquantes
    """
    nb_nan_tot = df.isna().sum().sum()
    nb_donnees_tot = np.prod(df.shape)
    nb_nan_pourcent = round((nb_nan_tot / nb_donnees_tot) * 100, 2)
    return nb_nan_tot, nb_donnees_tot, nb_nan_pourcent

In [5]:
def calcul_valeur_manquante_table(df):
    """
    Crée une table des valeurs manquantes par colonne dans le DataFrame.

    :param df: DataFrame à analyser
    :return: DataFrame contenant le nombre et le pourcentage de valeurs
    manquantes par colonne
    """
    nan_nbr = df.isnull().sum()
    nan_pourcent = nan_nbr / len(df) * 100
    table = pd.concat([nan_nbr, nan_pourcent.round(2)], axis=1)
    table.columns = ['Nombre de valeurs manquantes', '% de valeurs manquantes']
    return table

In [6]:
def valeur_manquante_text_graph(df, tableau, matrice):
    """
    Affiche le nombre de valeurs manquantes, un tableau de la répartition des
    valeurs manquantes par variable,
    et une matrice de répartition des valeurs manquantes par variable.

    :param df: DataFrame à analyser
    :param tableau: si True, affiche le tableau de répartition des valeurs
    manquantes
    :param matrice: si True, affiche la matrice de répartition des valeurs
    manquantes
    """
    # Affichage des valeurs manquantes
    nb_nan_tot, nb_donnees_tot, nb_nan_pourcent = calcul_valeur_manquante(df)
    print('Valeurs manquantes (Nan) : {0} / {1} données ({2} %)'.format(
        nb_nan_tot, nb_donnees_tot, nb_nan_pourcent
    ))

    # Affichage du tableau de répartition des valeurs manquantes
    table = calcul_valeur_manquante_table(df)
    nb_col = df.columns.size
    nb_col_nan = table[table['Nombre de valeurs manquantes'] != 0][
        'Nombre de valeurs manquantes'].size
    print(f'{nb_col_nan} / {nb_col} colonne(s) concernée(s)')

    # Affichage du tableau de répartition des valeurs manquantes
    if tableau:
        display(
            table[table['Nombre de valeurs manquantes'] != 0]
            .sort_values('% de valeurs manquantes', ascending=False)
            .style.background_gradient('YlGn')
        )

    # Affichage de la matrice de répartition des valeurs manquantes
    if matrice:
        print("-" * 100)
        print("Heatmap de visualisation des valeurs manquantes")
        msno.matrix(df)

In [7]:
# def calculer_metriques(
#     y_train,
#     y_prob,
#     elapsed_time_fit,
#     elapsed_time_predict
# ):
#     """
#     Calcule et renvoie plusieurs métriques de performance pour un modèle de
#     classification.

#     Paramètres:
#     y_train (array-like): Les vraies étiquettes.
#     y_prob (array-like): Les probabilités prédites par le modèle.
#     elapsed_time_fit (float): Le temps pris par le modèle pour s'entraîner.
#     elapsed_time_predict (float): Le temps pris par le modèle pour faire
#     des prédictions.

#     Retourne:
#     metrics (dict): Un dictionnaire contenant plusieurs métriques de
#     performance.
#     """
#     # Calcul des valeurs prédites
#     y_pred = (y_prob > 0.5)

#     # Calcul du score_metier
#     tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()
#     score_metier = fp + 10 * fn

#     # Log les métriques à MLflow
#     metrics = {}
#     metrics['score_metier'] = score_metier
#     metrics['accuracy'] = accuracy_score(y_train, y_pred)
#     metrics['AUC'] = roc_auc_score(y_train, y_prob)
#     metrics['f1_score'] = f1_score(y_train, y_pred)
#     metrics['temps_fit'] = elapsed_time_fit
#     metrics['temps_predict'] = elapsed_time_predict

#     return metrics

In [8]:
def plot_and_save_courbe_roc(y, y_prob, auc):
    """
    Trace et enregistre la courbe ROC.

    Paramètres
    ----------
    y : array-like
        Les vraies valeurs de la cible.
    y_prob : array-like
        Les probabilités prédites pour la classe positive.
    auc : float
        L'aire sous la courbe ROC.

    """
    fpr, tpr, thresholds = roc_curve(y, y_prob)
    plt.figure(figsize=(10, 7))
    plt.plot(fpr, tpr, label=f'AUC: {auc:.2f}')
    plt.title('Courbe ROC')
    plt.xlabel('Taux de faux positifs')
    plt.ylabel('Taux de vrai positifs')
    plt.legend(loc='lower right')
    plt.savefig("figure/courbe_roc.png")
    plt.close()

In [9]:
def define_preprocessor(X):
    """
    Définit un préprocesseur pour les caractéristiques numériques et
    catégorielles.

    Le préprocesseur est un ColumnTransformer qui applique différentes
    transformations
    aux caractéristiques numériques et catégorielles. Pour les
    caractéristiques catégorielles,
    les transformations sont : imputation des valeurs manquantes, encodage
    cible et mise à l'échelle.
    Pour les caractéristiques numériques, les transformations sont :
    imputation des valeurs manquantes
    et mise à l'échelle.

    Paramètres
    ----------
    X : DataFrame
        Le DataFrame contenant les caractéristiques à prétraiter.

    Retourne
    -------
    preprocessor : ColumnTransformer
        Le préprocesseur défini.
    """
    # Feature Categories
    cat_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(
                strategy='constant',
                fill_value='missing'
            )),
            ('target_encoder', TargetEncoder()),
            ('scaler', StandardScaler())
            ])

    # Feature Numbers
    num_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler()),
    ])

    # Création pipeline preprocess
    preprocessor = ColumnTransformer(transformers=[
            ('cat', cat_transformer, list(
                X.select_dtypes(exclude='number').columns
            )),
            ('num', num_transformer, list(
                X.select_dtypes(include='number').columns
            )),
    ])

    return preprocessor

In [10]:
def define_pipeline(preprocessor, random_seed, modele):
    """
    Définit un pipeline pour le prétraitement et le modèle.

    Le pipeline est une séquence d'étapes où chaque étape est une paire
    (nom, transformateur ou estimateur).
    Les étapes sont exécutées dans l'ordre.

    Paramètres
    ----------
    preprocessor : Transformer
        Le préprocesseur à utiliser pour le prétraitement des données.
    modele : Estimator
        Le modèle à utiliser pour l'apprentissage.

    Retourne
    -------
    pipe : Pipeline
        Le pipeline défini.
    """
    # Création pipeline
    pipe = Pipeline(steps=[
            ("preprocess", preprocessor),
            ('sampling', SMOTE(random_state=random_seed)),
            ("modele", modele)
        ]
    )

    return pipe

In [11]:
def surligne_meilleur(s):
    """
    Cette fonction met en évidence la valeur maximale pour 'AUC', 'accuracy',
    'f1_score' et la valeur minimale pour 'score_metier', 'temps_fit' et
    'temps_prediction' dans une série pandas.

    Paramètres :
    s (pd.Series) : Une série pandas contenant les données à mettre en avant.

    Retourne :
    list : Une liste de chaînes de caractères représentant les styles CSS pour
    chaque élément de la série. Les valeurs maximales sont mises en
    évidence avec un fond vert et les valeurs minimales avec un fond bleu.

    """
    # Séries de booléens pour les valeurs maximales et minimales
    is_max = pd.Series([False] * len(s), index=s.index)
    is_min = pd.Series([False] * len(s), index=s.index)

    # Métriques à mettre en évidence (max et min)
    max_metrics = ['AUC', 'accuracy', 'f1_score']
    min_metrics = ['score_metier', 'temps_fit', 'temps_prediction']

    # SI la métrique est dans la liste des métriques min
    if s.name in max_metrics:
        is_max = s == s.max()

    # SI la métrique est dans la liste des métriques max
    elif s.name in min_metrics:
        is_min = s == s.min()

    return [
        'background-color: green' if v
        else 'background-color: blue' if w else '' for v,
        w in zip(is_max, is_min)
    ]

In [12]:
def metier_metric_erreur_predict(y_true, y_pred):
    """
    Calcule la metric du score métier du cout de l'erreur de prédiction.
    (l'erreur de prédiction 'False Negative' est 10 fois plus couteuse
    que 'False Positive')

    Args:
        y_true (array-like): Les vraies valeurs de la cible.
        y_pred (array-like): Les valeurs prédites.

    Returns:
        int: La metrique du score métier.
    """
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    metier_score_erreur_predict = fp + 10 * fn
    return metier_score_erreur_predict

In [13]:
def process_cv_results(
    cv_results,
    important_columns,
    sort_column='rank_test_score',
    drop_prefixes=['std_', 'params'],
    ascending_order=True
):
    """
    Traite les résultats d'une recherche d'hyperparamètres en supprimant
    les colonnes inutiles et en réorganisant les colonnes importantes.

    Paramètres
    ----------
    cv_results : dict
        Les résultats de la recherche d'hyperparamètres, généralement obtenus
        avec `search.cv_results_`.
    important_columns : list of str
        Les noms des colonnes importantes à mettre en premier dans le
        DataFrame traité.
    sort_column : str, optional
        Le nom de la colonne à utiliser pour trier le DataFrame. Par défaut,
        'rank_test_score'.
    drop_prefixes : list of str, optional
        Les préfixes des noms de colonnes à supprimer du DataFrame. Par
        défaut, ['std_', 'params'].
    ascending_order : bool, optional
        Si True, trier le DataFrame en ordre croissant. Sinon, trier en ordre
        décroissant. Par défaut, True.

    Retourne
    -------
    sorted_clean_df : DataFrame
        Le DataFrame traité avec les colonnes inutiles supprimées et les
        colonnes importantes réorganisées.
    """
    # Convertir les résultats en DataFrame
    cv_results_df = pd.DataFrame(cv_results)

    # Trier le DataFrame par la colonne spécifiée
    sorted_df = cv_results_df.sort_values(
        by=sort_column,
        ascending=ascending_order
    )

    # Supprimer les col dont le nom commence par l'un des préfixes spécifiés
    drop_cols = [
        col for col in sorted_df.columns
        if any(col.startswith(prefix) for prefix in drop_prefixes)
    ]
    sorted_clean_df = sorted_df.drop(columns=drop_cols)

    # Créer une nouvelle liste de col
    new_columns = important_columns + [
        col for col in sorted_clean_df.columns if col not in important_columns
    ]

    # Réindexer le DataFrame avec la nouvelle liste de col
    sorted_clean_df = sorted_clean_df[new_columns]

    return sorted_clean_df

In [14]:
def enleve_prefix_des_params(params, prefix='modele__'):
    """
    Supprime un préfixe spécifié des clés d'un dictionnaire de paramètres.

    Paramètres
    ----------
    params : dict
        Le dictionnaire de paramètres dont les clés peuvent contenir le
        préfixe à supprimer.
    prefix : str, optional
        Le préfixe à supprimer des clés du dictionnaire de paramètres.
        Par défaut, 'modele__'.

    Retourne
    -------
    new_params : dict
        Le dictionnaire de paramètres avec le préfixe supprimé des clés.
    """
    new_params = {k.replace(prefix, ''): v for k, v in params.items()}
    return new_params

# <span style='background:grey'>Importation des donnees</span>

Importation des fichiers CSV avec Pandas

In [15]:
# Definir le chemin des datasets
chemin = "./data/cleaned/"

# Obtenir une liste de tous les fichiers dans le repertoire
fichiers = os.listdir(chemin)

# Initialiser un dictionnaire vide
data_clean = {}

# Chemin du fichier pickle
pickle_path = chemin + 'data_cleaned.pickle'

# Vérifier si le fichier pickle existe
if os.path.exists(pickle_path):

    # Charger les données à partir du fichier pickle
    with open(pickle_path, 'rb') as handle:
        data_clean = pickle.load(handle)
    print("===> Données chargées à partir du fichier pickle <====\n")
else:
    print("===> Liste des fichiers importes: <====\n")

    # Boucle sur chaque fichier
    for fichier in fichiers:

        # Traiter uniquement les fichiers .csv
        if fichier.endswith(".csv"):
            print(fichier)

            # Lire le fichier csv et le stocker dans le dictionnaire
            data_clean[fichier] = pd.read_csv(
                chemin + fichier,
                sep=",",
                encoding="latin-1",
                index_col=0
            )

    # Sauvegarder les données dans un fichier pickle
    with open(pickle_path, 'wb') as handle:
        pickle.dump(data_clean, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("\n===> Données sauvegardées dans un fichier pickle <====\n")

===> Données chargées à partir du fichier pickle <====



In [16]:
# isoler le contenu du fichier et afficher le contenu
nom_fichier = "application_train_cleaned.csv"
train_df = data_clean[nom_fichier]
display(train_df)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,LIVINGAREA_AVG,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Spouse partner,NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,...,PREV_NAME_TYPE_SUITE_Other_B_MEAN,PREV_NAME_TYPE_SUITE_Spouse partner_MEAN,PREV_NAME_TYPE_SUITE_Unaccompanied_MEAN,PREV_NAME_CLIENT_TYPE_New_MEAN,PREV_NAME_CLIENT_TYPE_Refreshed_MEAN,PREV_NAME_CLIENT_TYPE_XNA_MEAN,PREV_NAME_GOODS_CATEGORY_Additional Service_MEAN,PREV_NAME_GOODS_CATEGORY_Animals_MEAN,PREV_NAME_GOODS_CATEGORY_Audio Video_MEAN,PREV_NAME_GOODS_CATEGORY_Auto Accessories_MEAN,PREV_NAME_GOODS_CATEGORY_Clothing and Accessories_MEAN,PREV_NAME_GOODS_CATEGORY_Computers_MEAN,PREV_NAME_GOODS_CATEGORY_Construction Materials_MEAN,PREV_NAME_GOODS_CATEGORY_Consumer Electronics_MEAN,PREV_NAME_GOODS_CATEGORY_Direct Sales_MEAN,PREV_NAME_GOODS_CATEGORY_Education_MEAN,PREV_NAME_GOODS_CATEGORY_Fitness_MEAN,PREV_NAME_GOODS_CATEGORY_Furniture_MEAN,PREV_NAME_GOODS_CATEGORY_Gardening_MEAN,PREV_NAME_GOODS_CATEGORY_Homewares_MEAN,PREV_NAME_GOODS_CATEGORY_House Construction_MEAN,PREV_NAME_GOODS_CATEGORY_Insurance_MEAN,PREV_NAME_GOODS_CATEGORY_Jewelry_MEAN,PREV_NAME_GOODS_CATEGORY_Medical Supplies_MEAN,PREV_NAME_GOODS_CATEGORY_Medicine_MEAN,PREV_NAME_GOODS_CATEGORY_Mobile_MEAN,PREV_NAME_GOODS_CATEGORY_Office Appliances_MEAN,PREV_NAME_GOODS_CATEGORY_Other_MEAN,PREV_NAME_GOODS_CATEGORY_Photo Cinema Equipment_MEAN,PREV_NAME_GOODS_CATEGORY_Sport and Leisure_MEAN,PREV_NAME_GOODS_CATEGORY_Tourism_MEAN,PREV_NAME_GOODS_CATEGORY_Vehicles_MEAN,PREV_NAME_GOODS_CATEGORY_Weapon_MEAN,PREV_NAME_PORTFOLIO_Cars_MEAN,PREV_NAME_PRODUCT_TYPE_XNA_MEAN,PREV_NAME_PRODUCT_TYPE_walk in_MEAN,PREV_NAME_PRODUCT_TYPE_x sell_MEAN,PREV_CHANNEL_TYPE_AP Cash loan _MEAN,PREV_CHANNEL_TYPE_Channel of corporate sales_MEAN,PREV_CHANNEL_TYPE_Contact center_MEAN,PREV_CHANNEL_TYPE_Country wide_MEAN,PREV_CHANNEL_TYPE_Credit and cash offices_MEAN,PREV_CHANNEL_TYPE_Regional Local_MEAN,PREV_CHANNEL_TYPE_Stone_MEAN,PREV_NAME_SELLER_INDUSTRY_Auto technology_MEAN,PREV_NAME_SELLER_INDUSTRY_Construction_MEAN,PREV_NAME_SELLER_INDUSTRY_Consumer electronics_MEAN,PREV_NAME_SELLER_INDUSTRY_Industry_MEAN,PREV_NAME_SELLER_INDUSTRY_Jewelry_MEAN,PREV_NAME_SELLER_INDUSTRY_MLM partners_MEAN,PREV_NAME_SELLER_INDUSTRY_Tourism_MEAN,PREV_NAME_YIELD_GROUP_XNA_MEAN,PREV_NAME_YIELD_GROUP_high_MEAN,PREV_NAME_YIELD_GROUP_low_action_MEAN,PREV_NAME_YIELD_GROUP_low_normal_MEAN,PREV_NAME_YIELD_GROUP_middle_MEAN,PREV_PRODUCT_COMBINATION_Card X Sell_MEAN,PREV_PRODUCT_COMBINATION_Cash Street high_MEAN,PREV_PRODUCT_COMBINATION_Cash Street low_MEAN,PREV_PRODUCT_COMBINATION_Cash Street middle_MEAN,PREV_PRODUCT_COMBINATION_Cash X Sell high_MEAN,PREV_PRODUC

In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307488 entries, 0 to 307510
Columns: 430 entries, SK_ID_CURR to APPROVED_CNT_PAYMENT_SUM
dtypes: float64(262), int64(168)
memory usage: 1011.1 MB


## <span style='background:red'>DEV debut</span>

In [18]:
# Garde 1 fraction (10%) des données pour accélérer le temps de calcul en DEV
train_df = train_df.sample(frac=0.1, random_state=random_seed)

## <span style='background:red'>DEV fin</span>

# <span style='background:blue'>Etape 1 : Feature engineering</span>

## <span style='background:green'>1/ Séparation features explicatives et feature cible</span>

Nous séparons le dataset en 2 jeux de données :
- X : les features explicatives (sans l'identifiant de prêt)
- y : la feature cible, indique si le client a eu des difficultés à rembourser son prêt (1) ou non (0)

In [19]:
X = train_df.drop(columns=["TARGET", "SK_ID_CURR"])
y = train_df["TARGET"]

Nous vérifions le dataframe des features explicatives.

In [20]:
display(X)

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,LIVINGAREA_AVG,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Spouse partner,NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,...,PREV_NAME_TYPE_SUITE_Other_B_MEAN,PREV_NAME_TYPE_SUITE_Spouse partner_MEAN,PREV_NAME_TYPE_SUITE_Unaccompanied_MEAN,PREV_NAME_CLIENT_TYPE_New_MEAN,PREV_NAME_CLIENT_TYPE_Refreshed_MEAN,PREV_NAME_CLIENT_TYPE_XNA_MEAN,PREV_NAME_GOODS_CATEGORY_Additional Service_MEAN,PREV_NAME_GOODS_CATEGORY_Animals_MEAN,PREV_NAME_GOODS_CATEGORY_Audio Video_MEAN,PREV_NAME_GOODS_CATEGORY_Auto Accessories_MEAN,PREV_NAME_GOODS_CATEGORY_Clothing and Accessories_MEAN,PREV_NAME_GOODS_CATEGORY_Computers_MEAN,PREV_NAME_GOODS_CATEGORY_Construction Materials_MEAN,PREV_NAME_GOODS_CATEGORY_Consumer Electronics_MEAN,PREV_NAME_GOODS_CATEGORY_Direct Sales_MEAN,PREV_NAME_GOODS_CATEGORY_Education_MEAN,PREV_NAME_GOODS_CATEGORY_Fitness_MEAN,PREV_NAME_GOODS_CATEGORY_Furniture_MEAN,PREV_NAME_GOODS_CATEGORY_Gardening_MEAN,PREV_NAME_GOODS_CATEGORY_Homewares_MEAN,PREV_NAME_GOODS_CATEGORY_House Construction_MEAN,PREV_NAME_GOODS_CATEGORY_Insurance_MEAN,PREV_NAME_GOODS_CATEGORY_Jewelry_MEAN,PREV_NAME_GOODS_CATEGORY_Medical Supplies_MEAN,PREV_NAME_GOODS_CATEGORY_Medicine_MEAN,PREV_NAME_GOODS_CATEGORY_Mobile_MEAN,PREV_NAME_GOODS_CATEGORY_Office Appliances_MEAN,PREV_NAME_GOODS_CATEGORY_Other_MEAN,PREV_NAME_GOODS_CATEGORY_Photo Cinema Equipment_MEAN,PREV_NAME_GOODS_CATEGORY_Sport and Leisure_MEAN,PREV_NAME_GOODS_CATEGORY_Tourism_MEAN,PREV_NAME_GOODS_CATEGORY_Vehicles_MEAN,PREV_NAME_GOODS_CATEGORY_Weapon_MEAN,PREV_NAME_PORTFOLIO_Cars_MEAN,PREV_NAME_PRODUCT_TYPE_XNA_MEAN,PREV_NAME_PRODUCT_TYPE_walk in_MEAN,PREV_NAME_PRODUCT_TYPE_x sell_MEAN,PREV_CHANNEL_TYPE_AP Cash loan _MEAN,PREV_CHANNEL_TYPE_Channel of corporate sales_MEAN,PREV_CHANNEL_TYPE_Contact center_MEAN,PREV_CHANNEL_TYPE_Country wide_MEAN,PREV_CHANNEL_TYPE_Credit and cash offices_MEAN,PREV_CHANNEL_TYPE_Regional Local_MEAN,PREV_CHANNEL_TYPE_Stone_MEAN,PREV_NAME_SELLER_INDUSTRY_Auto technology_MEAN,PREV_NAME_SELLER_INDUSTRY_Construction_MEAN,PREV_NAME_SELLER_INDUSTRY_Consumer electronics_MEAN,PREV_NAME_SELLER_INDUSTRY_Industry_MEAN,PREV_NAME_SELLER_INDUSTRY_Jewelry_MEAN,PREV_NAME_SELLER_INDUSTRY_MLM partners_MEAN,PREV_NAME_SELLER_INDUSTRY_Tourism_MEAN,PREV_NAME_YIELD_GROUP_XNA_MEAN,PREV_NAME_YIELD_GROUP_high_MEAN,PREV_NAME_YIELD_GROUP_low_action_MEAN,PREV_NAME_YIELD_GROUP_low_normal_MEAN,PREV_NAME_YIELD_GROUP_middle_MEAN,PREV_PRODUCT_COMBINATION_Card X Sell_MEAN,PREV_PRODUCT_COMBINATION_Cash Street high_MEAN,PREV_PRODUCT_COMBINATION_Cash Street low_MEAN,PREV_PRODUCT_COMBINATION_Cash Street middle_MEAN,PRE

Nous vérifions le dataframe de la feature cible.

In [21]:
display(y)

# verifier pas de nan
print(f"Nombre de Nan : {y.isna().sum()}")

113319    0
221599    0
11048     0
278721    0
25607     0
         ..
26118     0
145547    0
297476    0
81359     0
146040    0
Name: TARGET, Length: 30749, dtype: int64

Nombre de Nan : 0


## <span style='background:green'>2/ Séparation des jeux d'entrainement et de test</span>

Nous séparons les 2 dataframes en 2 jeux : 
- un jeu d'entrainement (X et y) *contenant 75% des données*
- un jeu de test (X et y) *contenant 25% des données*

In [22]:
# Séparation des données en données d'entrainement et de test (75% / 25%)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=random_seed
)

# Affichage des dimensions des données
print(f"X_train fait {X_train.shape} et y_train fait {y_train.shape}.")
print(f"X_test fait {X_test.shape} et y_test fait {y_test.shape}.")

X_train fait (23061, 428) et y_train fait (23061,).
X_test fait (7688, 428) et y_test fait (7688,).


## <span style='background:green'>3/ Imputation</span>

Preprocessor contient: 
- l'imputation des valeurs manquantes par ColumnTransformer:
    - catégorielle : 
      - Imputation 'constant' des 'valeurs manquantes'
      - Target encoding
      - Standard Scaler
    - numérique : 
      - Imputation 'median'
      - Standard Scaler


In [23]:
# Compter le nombre total de valeurs NaN dans le DataFrame
num_nan = X_train.isna().sum().sum()
print(f"Nombre total de valeurs NaN : {num_nan}")

# Intancier le préprocesseur
preprocessor = define_preprocessor(X_train)

Nombre total de valeurs NaN : 564166


## <span style='background:green'>4/ Pipeline général</span>

Le pipeline général contient :
- 'preprocessor' : imputation des valeurs manquantes
- 'sampling' : gestion du déséquilibre des classes
- 'model' : modèle de classification

## <span style='background:green'>5/ Validation croisée startifiée</span>


In [24]:
#  Validation croisée stratifiée
cv_skf = StratifiedKFold(n_splits=5)

## <span style='background:green'>6/ Score métier</span>

Nous calculons la metric du score métier du cout de l'erreur de prédiction.
    (l'erreur de prédiction 'False Negative' est 10 fois plus couteuse
    que 'False Positive')

In [25]:
metier_score_erreur_predict = make_scorer(
    metier_metric_erreur_predict,
    greater_is_better=False
)

## <span style='background:green'>7/ Reposit MLflow</span>


In [26]:
# Déterminer l'URI de suivi MLflow
mlflow.set_tracking_uri("file:./mlruns")

## <span style='background:green'>8/ Feature importantes pour ranking</span>


In [27]:
# Definir les premières features à afficher pour cv_results
important_col_cv_results = [
    'rank_test_score',
    'mean_test_score',
    'params'
]

# <span style='background:blue'>Etape 2 : Modèle naïf</span>

## <span style='background:green'>1/ Version local</span>

In [30]:
# # Instancier le pipeline du modèle
# model = define_pipeline(
#     preprocessor,
#     random_seed,
#     DummyClassifier(random_state=random_seed)
# )

# # Définir les hyperparamètres à tester
# param_grid = {
#     'modele__strategy': ['most_frequent', 'prior', 'stratified', 'uniform']
# }

# # Validation croisée avec recherche d'hyperparamètres (bayesienne)
# bayes_search = BayesSearchCV(
#     model,
#     param_grid,
#     cv=cv_skf,
#     scoring=metier_score_erreur_predict,
#     n_jobs=-1,
#     random_state=random_seed,
#     n_iter=10,
# )
# bayes_search.fit(X_train, y_train)

# # Résultats de la validation croisée (bayesienne)
# sorted_clean_df = process_cv_results(
#     bayes_search.cv_results_,
#     important_col_cv_results,
#     sort_column='rank_test_score',
#     drop_prefixes=['std_', 'param_']
# )
# display(sorted_clean_df)

# # Meilleurs hyperparamètres : suppression du préfixe et affichage
# best_params = enleve_prefix_des_params(bayes_search.best_params_)
# print(f"Best parameters: {bayes_search.best_params_}")

# # Instancier best_model avec les meilleurs hyperparamètres
# model_best = define_pipeline(
#     preprocessor,
#     random_seed,
#     DummyClassifier(**best_params, random_state=random_seed)
# )

# # Entainer le best_model sur les données d'entraînement
# model_best.fit(X_train, y_train)

## <span style='background:green'>2/ Version MLFLOW</span>

In [33]:
# Créer un nouvel experiment MLflow et l'activer
mlflow.create_experiment("Comparaison_models")
mlflow.set_experiment("Comparaison_models")

# Lancez un run MLflow pour le paramètre donné
with mlflow.start_run(run_name="dummy_classifier"):

    # Enregistrement de plusieurs paramètres sous forme d'un dict
    params = {}
    params['TRAIN_SIZE'] = len(X_train)
    params['SEED'] = random_seed
    mlflow.log_params(params)

    # Instancier le pipeline du modèle
    model = define_pipeline(
        preprocessor,
        random_seed,
        DummyClassifier(random_state=random_seed)
    )

    # Définir les hyperparamètres à tester
    param_grid = {
        'modele__strategy': ['most_frequent', 'prior', 'stratified', 'uniform']
    }

    # Validation croisée avec recherche d'hyperparamètres (bayesienne)
    bayes_search = BayesSearchCV(
        model,
        param_grid,
        cv=cv_skf,
        scoring=metier_score_erreur_predict,
        n_jobs=-1,
        random_state=random_seed,
        n_iter=10,
    )
    bayes_search.fit(X_train, y_train)

    # Résultats de la validation croisée (bayesienne)
    cv_results_clean_df = process_cv_results(
        bayes_search.cv_results_,
        important_col_cv_results,
        sort_column='rank_test_score',
        drop_prefixes=['std_', 'param_']
    )
    display(cv_results_clean_df)

    # Meilleurs hyperparamètres : suppression du préfixe et affichage
    best_params = enleve_prefix_des_params(bayes_search.best_params_)
    print(f"Best parameters: {best_params}")

    # Instancier best_model avec les meilleurs hyperparamètres
    model_best = define_pipeline(
        preprocessor,
        random_seed,
        DummyClassifier(**best_params, random_state=random_seed)
    )

    # Mesurer le temps de traitement de fit
    start_time_fit = time.time()
    model_best.fit(X_train, y_train)
    elapsed_time_fit = time.time() - start_time_fit

    # Sauvegarder le df des resultats dans un fichier CSV
    cv_results_clean_df.to_csv('results/cv_results_clean_df.csv', index=False)

    # Enregistrer le temps de traitement de fit comme une métrique
    mlflow.log_metric("temps_fit", elapsed_time_fit)

    # Enregistrer le fichier CSV comme un artefact
    mlflow.log_artifact('results/cv_results_clean_df.csv')

    # Log le modèle à MLflow
    mlflow.sklearn.log_model(model_best, "model_dummy_best")

# Nettoyer les objets
print("Objets nettoyés :")
gc.collect()

,rank_test_score,mean_test_score,params,mean_fit_time,mean_score_time,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
0,1,-3708.0,{'modele__strategy': 'prior'},2.13,0.11,-3710.0,-3700.0,-3710.0,-3710.0,-3710.0
2,1,-3708.0,{'modele__strategy': 'prior'},2.05,0.10,-3710.0,-3700.0,-3710.0,-3710.0,-3710.0
9,1,-3708.0,{'modele__strategy': 'most_frequent'},2.16,0.13,-3710.0,-3700.0,-3710.0,-3710.0,-3710.0
1,4,-3903.8,{'modele__strategy': 'uniform'},2.02,0.11,-3798.0,-3788.0,-3919.0,-4040.0,-3974.0
8,4,-3903.8,{'modele__strategy': 'uniform'},2.30,0.11,-3798.0,-3788.0,-3919.0,-4040.0,-3974.0
3,6,-4054.6,{'modele__strategy': 'stratified'},2.20,0.12,-4086.0,-4075.0,-4096.0,-3931.0,-4085.0
4,6,-4054.6,{'modele__strategy': 'stratified'},2.02,0.10,-4086.0,-4075.0,-4096.0,-3931.0,-4085.0
5,6,-4054.6,{'modele__strategy': 'stratified'},2.33,0.13,-4086.0,-4075.0,-4096.0,-3931.0,-4085.0
6,6,-4054.6,{'modele__strategy': 'stratified'},2.41,0.12,-4086.0,-4075.0,-4096.0,-3931.0,-4085.0
7,6,-4054.6,{'modele__strategy': 'stratified'},2.30,0.12,-4086.0,-4075.0,-4096.0,-3931.0,-4085.0


Best parameters: OrderedDict([('modele__strategy', 'prior')])
Objets nettoyés :


737

Nous lançons la commande mlflow ui pour visualiser les résultats de l'expérience.

In [34]:
# !mlflow ui

Ouvrez maintenant un navigateur web et ouvrez une fenêtre vers l'url http://localhost:5000/

# <span style='background:blue'>Etape 3 : Regression logistique</span>

## <span style='background:green'>1/ Version local</span>

## <span style='background:green'>2/ Version MLFLOW</span>

# <span style='background:blue'>Etape 4 : Random forest</span>

## <span style='background:green'>1/ Version local</span>

In [ ]:
# Définir les hyperparamètres à tester
param_grid = {
    'modele__max_depth': (1, 10),
    'modele__min_samples_split': (2, 10),
    'modele__n_estimators': (50, 200)
}

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   35.1s finished


Cross-validation scores: [0.91675699 0.91934085 0.91760624 0.91717259 0.91847355]
Best parameters: OrderedDict([('modele__max_depth', 10), ('modele__min_samples_split', 8), ('modele__n_estimators', 181)])


0.9185744016649323

## <span style='background:green'>2/ Version MLFLOW</span>

# <span style='background:blue'>Etape 5 : Light GBM</span>

## <span style='background:green'>1/ Version local</span>

## <span style='background:green'>2/ Version MLFLOW</span>

**Définir la stratégie d’élaboration d’un modèle d’apprentissage supervisé et sélectionner et entraîner des modèles adaptés à une problématique métier afin de réaliser une analyse prédictive.**

~~CE1 Les variables catégorielles identifiées ont été transformées en fonction du besoin (par exemple via OneHotEncoder ou TargetEncoder).~~

~~CE2 Vous avez a créé de nouvelles variables à partir de variables existantes.~~

CE3 Vous avez réalisé des transformations mathématiques lorsque c'est requis pour transformer les distributions de variables. 

~~CE4 Vous avez normalisé les variables lorsque c'est requis.~~

CE5 Vous avez défini sa stratégie d’élaboration d’un modèle pour répondre à un besoin métier. Cela signifie dans ce projet que : l’étudiant a présenté son approche méthodologique de modélisation dans son support de présentation pendant la soutenance et est capable de répondre à des questions à ce sujet, si elles lui sont posées.

~~CE6 Vous avez choisi la ou les variables cibles pertinentes.~~

CE7 Vous avez vérifié qu'il n’y a pas de problème de data leakage (c'est-à-dire, des variables trop corrélées à la variable cible et inconnues a priori dans les données en entrée du modèle).

CE8 Vous avez testé plusieurs algorithmes de façon cohérente, en partant des plus simples vers les plus complexes (au minimum un linéaire et un non linéaire).

**Évaluer les performances des modèles d’apprentissage supervisé selon différents critères (scores, temps d'entraînement, etc.) en adaptant les paramètres afin de choisir le modèle le plus performant pour la problématique métier.**

~~CE1 Vous avez choisi une métrique adaptée pour évaluer la performance d'un algorithme (par exemple : R2 ou RMSE en régression, accuracy ou AUC en classification, etc.). Dans le cadre de ce projet, cela signifie que : Vous avez mis en oeuvre un score métier pour évaluer les modèles et optimiser les hyperparamètres, qui prend en compte les spécificités du contexte, en particulier le fait que le coût d’un faux négatif et d’un faux positif sont sensiblement différents.~~

CE2 Vous avez exploré d'autres indicateurs de performance que le score pour comprendre les résultats (coefficients des variables en fonction de la pénalisation, visualisation des erreurs en fonction des variables du modèle, temps de calcul...).

~~CE3 Vous avez séparé les données en train/test pour les évaluer de façon pertinente et détecter l'overfitting.~~

~~CE4 Vous avez mis en place un modèle simple de référence pour évaluer le pouvoir prédictif du modèle choisi (dummyRegressor ou dummyClassifier).~~

CE5 Vous avez pris en compte dans sa démarche de modélisation l'éventuel déséquilibre des classes (dans le cas d'une classification).

CE6 Vous avez optimisé les hyper-paramètres pertinents dans les différents algorithmes.

CE7 Vous avez mis en place une validation croisée (via GridsearchCV, RandomizedSearchCV ou équivalent) afin d’optimiser les hyperparamètres et comparer les modèles. Dans le cadre de ce projet : 

- Une cross-validation du dataset train est réalisée,
- Un premier test de différentes valeurs d’hyperparamètres est réalisé sur chaque algorithme testé, et affiné pour l’algorithme final choisi,
- Tout projet présentant un score AUC anormalement élevé, démontrant de l’overfitting dans le GrisSearchCV, sera invalidé (il ne devrait pas être supérieur au meilleur de la compétition Kaggle : 0.82).

CE8 Vous avez présenté l'ensemble des résultats en allant des modèles les plus simples aux plus complexes. Vous avez justifié le choix final de l'algorithme et des hyperparamètres.

CE9 Vous avez réalisé l’analyse de l’importance des variables (feature importance) globale sur l’ensemble du jeu de données et locale sur chaque individu du jeu de données.

**Définir et mettre en œuvre un pipeline d’entraînement des modèles, avec centralisation du stockage des modèles et formalisation des résultats et mesures des différentes expérimentations réalisées, afin d’industrialiser le projet de Machine Learning.**

CE1 Vous avez mis en oeuvre un pipeline d’entraînement des modèles reproductible.

CE2 Vous avez sérialisé et stocké les modèles créés dans un registre centralisé afin de pouvoir facilement les réutiliser.

CE3 Vous avez formalisé des mesures et résultats de chaque expérimentation, afin de les analyser et de les comparer